# signac-flow minimal example (with HOOMD-blue)

## About

This notebook contains a minimal example for running a signac flow project from scratch.
The example demonstrates how to compare an ideal gas with a lennard jones fluid by calculating a p-V phase diagram.

## Author

Carl Simon Adorf

## Before you start

Make sure you installed signac, signac-flow and HOOMD-blue, e.g., with:

```
conda install -c glotzer  signac
conda install -c glotzer  signac-flow
conda install -c glotzer  hoomd
```

In [1]:
import signac
import flow
import hoomd
import numpy as np

# Display plots within the notebook
from matplotlib import pyplot as plt
%matplotlib inline

# Initialize the HOOMD-blue execution context
if hoomd.context.exec_conf is None:
    hoomd.context.initialize('')


# Enter the signac project directory
% cd projects/flow-hoomd-example

HOOMD-blue v2.0.2 DOUBLE HPMC_MIXED SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX 
Compiled: 08/09/16
Copyright 2009-2016 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
/Users/csadorf/local/signac-examples/projects/hoomd-example


We want to generate pressure-volume phase diagram for a Lennard-Jones fluid with molecular dynamics (MD) using the general-purpose simulation toolkit HOOMD-blue (http://glotzerlab.engin.umich.edu/hoomd-blue/).

We start by defining two functions, one for the initialization of our simulation and one for the actual execution.

In [2]:
from math import ceil

def init(N):
    with hoomd.context.SimulationContext():
        n = ceil(pow(N, 1/3))
        assert n**3 == N
        hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=1.0), n=n)
        hoomd.dump.gsd('init.gsd', period=None, group=hoomd.group.all())

def sample_lj(N, sigma, seed, kT, tau, p, tauP, steps, r_cut):
    from hoomd import md
    with hoomd.context.SimulationContext():
        hoomd.init.read_gsd('init.gsd', restart='restart.gsd')
        group = hoomd.group.all()
        hoomd.dump.gsd('restart.gsd', truncate=True, period=100, phase=0, group=group)
        lj = md.pair.lj(r_cut=r_cut, nlist=md.nlist.cell())
        lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=sigma)
        md.integrate.mode_standard(dt=0.005)
        md.integrate.npt(group=group, kT=kT, tau=tau, P=p, tauP=tauP)
        hoomd.analyze.log('dump.log', ['volume'], 100, phase=0)
        hoomd.run_upto(steps)

We want to use **signac** to manage our simulation data and bundle all operations modifying our data space in a `Operations` class. This will make it easier later to execute these operations *by name*.

The `estimate` operation stores an ideal gas estimation of the volume for the given system.
The `sample` operation actually executes the MD simulation as defined in the previous cell.

In [ ]:
class Operations:
    
    def estimate(self, job):
        sp = job.statepoint()
        job.document['V'] = sp['N'] * sp['kT'] / sp['p']

    def sample(self, job):
        with job:
            sample_lj(steps=5000, ** job.statepoint())

Now that we have defined all *operations* it's time to embed them into a general *workflow*.
For this purpose we specialize a `flow.FlowProject`.

The workflow is controlled by two core functions: `classify()` and `next_operation()`:
 - The `classify()` function allows us to *label* our jobs and get a good overview of the project *status*. This is especially important once the data space becomes larger and more complex and operations more expensive.
 - The `next_operation()` functions helps to automate the workflow by identifying the next required operation for each job.

In [ ]:
class MyProject(flow.FlowProject):
    
    def classify(self, job):
        yield 'init'
        if 'V' in job.document:
            yield 'estimated'
        if job.isfile('dump.log'):
            yield 'sampled'
    
    def next_operation(self, job):
        labels = set(self.classify(job))
        if 'V' not in job.document:
            return 'estimate'
        if 'sampled' not in labels:
            return 'sample'

project = MyProject()

Now it's time to actually generate some data! Let's initialize the data space!


In [ ]:
# Uncomment the following two lines if you want to start over!
# for job in project.find_jobs():
#     job.remove()

for p in np.linspace(0.5, 5.0, 10):
    sp = dict(N=1728, sigma=1.0, seed=42, kT=1.0, p=p, tau=1.0, tauP=1.0, r_cut=2.5)
    with project.open_job(sp):
        init(N=sp['N'])

The `print_status()` function allows to get a quick overview of our project's *status*:

In [ ]:
project.print_status(detailed=True, parameters=['p'])

The next cell will attempt to execute all operations by cycling through jobs and operations until no *next operations* are defined anymore.

We limit the max. number of cycles to prevent accidental infinite loops, the number of cycles is arbitrary.

In [ ]:
for i in range(3):
    for job in project.find_jobs():
        for j in range(5):
            next_op = project.next_operation(job)
            if next_op is None:
                break
            print('execute', job, next_op)
            getattr(Operations(), next_op)(job)
            assert next_op != project.next_operation(job)
        else:
            raise RuntimeError("Reached max. # cycle limit!")

Let's double check the project status.

In [ ]:
project.print_status()

After running all operations we can make a brief examination of the collected data.

In [ ]:
def get_volume(job):
    log = np.genfromtxt(job.fn('dump.log'), names=True)
    N = len(log)
    return log[int(0.5 * N):]['volume'].mean(axis=0)

for job in project.find_jobs():
    print(job.statepoint()['p'], get_volume(job), job.document.get('V'))

For a better presentation of the results we need to aggregate all results and sort them by pressure.

In [ ]:
V = dict()
V_idg = dict()

for job in project.find_jobs():
    V[job.statepoint()['p']] = get_volume(job)
    V_idg[job.statepoint()['p']] = job.document['V']
    
p = sorted(V.keys())
V = [V[p_] for p_ in p]
V_idg = [V_idg[p_] for p_ in p]

plt.plot(p, V, label='LJ')
plt.plot(p, V_idg, label='idG')
plt.xlabel(r'pressure [$\epsilon / \sigma^3$]')
plt.ylabel(r'volume [$\sigma^3$]')
plt.legend()

As a a final step, we ca generate a index of our project data.
You can store this index in a variable or within a database, e.g., for search operations.

In [ ]:
import glotzformats as gf

formats = {
    '.*\.log': gf.formats.AnalysisLogFile
}
for doc in project.index(formats):
    print(doc)